# 卷积神经网络

 ## 1 简单的卷积网络
 ### 1.1 卷积模块

In [21]:
import torch
import math
import numpy as np
from torch import nn, optim
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

In [12]:
# 定义简单的卷积网络模型
class SimpleCNN(nn.Module):
    def __init__(self):            # 定义网络结构
        super().__init__()         # 输入 [batch_size,3,32,32] 3表示深度
        layer1 = nn.Sequential()   # 叠加第1个网络层，卷积层
        # (in_channels, out_channels, kernel_size,stride,padding)
        layer1.add_module('conv1', nn.Conv2d(32, 3, 1, padding=1))
        # get [batch_size, 32, 32, 32]
        layer1.add_module('relu1', nn.ReLU(True))
        layer1.add_module('pool1', nn.MaxPool2d(2,2))
        # get [batch_size, 32, 16, 16]
        self.layer1 = layer1
        
        layer2 = nn.Sequential()   # 定义第2个网络层
        layer2.add_module('conv2', nn.Conv2d(32, 64, 3, 1, padding=1))
        # get [batch_size, 64, 16, 16] 
        layer2.add_module('relu2', nn.ReLU(True))
        layer2.add_module('pool2', nn.MaxPool2d(2,2))
        # get [batch_size, 64, 8, 8]
        self.layer2 = layer2
        
        layer3 = nn.Sequential()   # 定义第3个网络层
        layer3.add_module('conv3', nn.Conv2d(64, 128, 3, 1, padding=1))
        # get [batch_size, 128, 8, 8]
        layer3.add_module('relu3', nn.ReLU(True))
        layer3.add_module('pool3', nn.MaxPool2d(2,2))
        # get [batch_size, 128, 4, 4]
        self.layer3 = layer3
        
        layer4 = nn.Sequential()   # 定义第4个网络层，全连接层
        layer4.add_module('fc1', nn.Linear(128*4*4, 512))
        # get [batch_size, 512]
        layer4.add_module('fc_relu1', nn.ReLU(True))
        layer4.add_module('fc2', nn.Linear(512, 64))
        # get [batch_size, 64]
        layer4.add_module('fc2_relu2', nn.ReLU(True))
        layer4.add_module('fc3', nn.Linear(64, 10))
        # get [batch_size, 10]
        self.layer4 = layer4
    def forward(self, x):
        conv1 = self.layer1(x)   # 前3层为卷积层
        conv2 = self.layer2(conv1)
        conv3 = self.layer3(conv2)
        # 注意：全连接前要把数据的维度将为两维，-1维度长*宽*深度
        fc_input = conv3.view(conv3.size(0), -1)  # reshape为 [batch_size, -1]
        fc_out = self.layer4(fc_input)
        return fc_out
model = SimpleCNN()
print(model)

SimpleCNN(
  (layer1): Sequential(
    (conv1): Conv2d(32, 3, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace)
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu2): ReLU(inplace)
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu3): ReLU(inplace)
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (fc1): Linear(in_features=2048, out_features=512, bias=True)
    (fc_relu1): ReLU(inplace)
    (fc2): Linear(in_features=512, out_features=64, bias=True)
    (fc2_relu2): ReLU(inplace)
    (fc3): Linear(in_features=64, out_features=10, bias=True)
  )
)


小结:
- 模型层顺序：CNN+BN+ReLU+Pooling，先激活再池化;FC+ReLU+Dropout
- 输出层不用激活函数

### 1.2 提取模型的层结构

|nn.Module属性|功能|实例|
|-|-|-|
|children()|返回下一级迭代器|self.layer1|
|modules()|返回所有模块迭代器|self.layer1.conv1|
|named_children()|返回模块的名称|其他功能同上|
|named_modules()|返回模块的名称|其他功能同上|

In [19]:
# 提取已有模型中的结构
new_model = nn.Sequential(*list(model.children())[:2])   # *代表使用可变参数
# print(new_model)
# list(model.children())[0]
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        print(m, m.kernel_size, m.out_channels)
        print('-----')

Conv2d(32, 3, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1)) (1, 1) 3
-----
Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) (3, 3) 64
-----
Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) (3, 3) 128
-----


In [46]:
# 提取模型中所有的卷积层
conv_model = nn.Sequential()
for layer in model.named_modules():    # layer[0]层名称, layer[1]层类型
#     print(layer)

    if isinstance(layer[1], nn.Conv2d):  
        print(layer)
        print('-'*50)
        conv_model.add_module(layer[0].split('.')[-1], layer[1])
print('1.新网络的结构:')
print(conv_model)

('layer1.conv1', Conv2d(32, 3, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1)))
--------------------------------------------------
('layer2.conv2', Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
--------------------------------------------------
('layer3.conv3', Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
--------------------------------------------------
1.新网络的结构:
Sequential(
  (conv1): Conv2d(32, 3, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


### 1.3 提取参数及初始化

|nn.Module属性|功能|
|-|-|
|parameters()|返回全部参数的迭代器|
|named_parameters()|返回模块的名称，其他功能同上|

In [64]:
# 提取模型的所有参数
for param in model.named_parameters():
    # param[0]:层名称.weight/bias  param[1]:参数值，权重是Variable
    print(param[0])
    print((param[1]).data.shape)
    print('-'*50)

layer1.conv1.weight
torch.Size([3, 32, 1, 1])
--------------------------------------------------
layer1.conv1.bias
torch.Size([3])
--------------------------------------------------
layer2.conv2.weight
torch.Size([64, 32, 3, 3])
--------------------------------------------------
layer2.conv2.bias
torch.Size([64])
--------------------------------------------------
layer3.conv3.weight
torch.Size([128, 64, 3, 3])
--------------------------------------------------
layer3.conv3.bias
torch.Size([128])
--------------------------------------------------
layer4.fc1.weight
torch.Size([512, 2048])
--------------------------------------------------
layer4.fc1.bias
torch.Size([512])
--------------------------------------------------
layer4.fc2.weight
torch.Size([64, 512])
--------------------------------------------------
layer4.fc2.bias
torch.Size([64])
--------------------------------------------------
layer4.fc3.weight
torch.Size([10, 64])
--------------------------------------------------
layer

In [73]:
# 提取参数并初始化????????
init = nn.init
for m in model.modules():  # 访问所有模块
    if isinstance(m, nn.Conv2d):
        init.normal_(m.weight.data)
        init.xavier_normal_(m.weight.data)
        init.kaiming_normal_(m.weight.data)
        m.bias.data.fill_(0)
    elif isinstance(m, nn.Linear):
        m.weight.data.normal_()

## 2 卷积神经网络实例
### 2.1 LeNet网络

conv层参数设置：[in_channels,out_channels,kernel_size,stride,padding]
$$W_{conv} = \frac{W_{input}-kernel\_size+2*padding}{stride}+1$$

maxpooling层参数设置：[kernel_size,stride,padding]

fc层参数设置：[in_channels,out_channels]

- 表格参数设置：[卷积核数/宽/高/深度] [池化宽/高] s:stride p:padding

|结构|参数设置|数据维度|
|-|-|-|
|input||[batch_size,3,32,32]|
|conv1|$[6\times5\times5\times3]$<br>s=1 p=0|[batch_size,6,28,28]|
|pool1|$[2\times2]$<br>s=2 p=0|[batch_size,6,14,14]|
|conv2|$[16\times5\times5\times6]$<br>s=1 p=0 |[batch_size,16,10,10]|
|pool2|$[2\times2]$<br>s=2 p=0|[batch_size,16,5,5]|
|fc1|$[(16*5*5)\times120]$|[batch_size,120]|
|fc2|$[120\times84]$|[batch_size,84]|
|fc3|$[84\times10]$|[batch_size,10]|
|--------|-------------------------------|--------------------------------|

In [48]:
# 定义网络结构
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()  # 输入维度[b,3,32,32]
        
        # 第一个卷积、池化模块
        layer1 = nn.Sequential()
        layer1.add_module('conv1', nn.Conv2d(3,6,5,1,padding=0))
        # get [b,6,28,28]
        layer1.add_module('pool1', nn.MaxPool2d(2,2))
        # get [b,6,14,14]
        self.layer1 = layer1
        
        # 第二个卷积、池化模块
        layer2 = nn.Sequential()
        layer2.add_module('conv2', nn.Conv2d(6,16,5,padding=0))
        # get [b,16,10,10]
        layer2.add_module('pool2', nn.MaxPool2d(2,2))
        # get [b,16,5,5]
        self.layer2 = layer2
        
        # 第三个全连接模块
        # 先将卷积的feature map reshape为[b,16*5*5]的形状
        layer3 = nn.Sequential()
        layer3.add_module('fc1', nn.Linear(16*5*5, 120))
        # get [b,120]
        layer3.add_module('fc2', nn.Linear(120, 84))
        # get [b, 84]
        layer3.add_module('fc3', nn.Linear(84, 10))
        # get [b, 10]
        self.layer3 = layer3
    def forward(self, x):
        conv1 = self.layer1(x)
        conv2 = self.layer2(conv1)
        fc_input = conv2.view(conv2.size(0), -1)
        fc_output = self.layer3(fc_input)
        return fc_output

In [60]:
%%time
LeNet_model = LeNet()
data_random = torch.Tensor(range(3*32*32))
input = data_random.reshape([1,3,32,32])
print('1.随机输入的数据:', input.shape)
output = LeNet_model(Variable(input))
print('2.模型输出:', output.shape)

1.随机输入的数据: torch.Size([1, 3, 32, 32])
2.模型输出: torch.Size([1, 10])
Wall time: 21 ms


### 2.2 AlexNet
2012年ILSVRC冠军网络

表格参数设置(与函数对应)：
- 卷积 [in_channels,out_channels,kernel_size,stride,padding]

- 池化 [kernel_size,stride,padding]

|结构|参数设置|数据维度|
|-|-|-|
|input||[batch_size,3,227,227]|
|conv1|[3,96,11,4,padding=0]  |[batch_size,96,55,55]|
|conv2|[96,256,5,1,padding=2] |[batch_size,256,55,55]|
|MaxPool|[3,2]                |[batch_size,256,27,27]|
|conv3|[256,384,3,1,padding=1]|[batch_size,384,27,27]|
|MaxPool|[3,2]                |[batch_size,384,13,13]|
|conv4|[384,384,3,1,padding=1]|[batch_size,384,13,13]|
|conv5|[384,256,3,1,padding=1]|[batch_size,256,13,13]|
|MaxPool|[3,2]                |[batch_size,256,6,6]|
|dense1|[256\*6\*6,4096]      |[batch_size,4096]|
|dense2|[4096,4096]           |[batch_size,4096]|
|dense3|[4096,1000]           |[batch_size,1000]|

In [57]:
# 定义AlexNet网络
class AlexNet(nn.Module):
    def __init__(self):
        super().__init__()    # 输入 [b,3,227,227]
        # 定义第1个卷积层
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 96, 11, 4, 0),    # get [b,96,55,55]
            nn.ReLU(True)
        )
        # 定义第2个卷积层
        self.conv2 = nn.Sequential(
            nn.Conv2d(96, 256, 5, 1, 2),   # get [b,256,55,55]
            nn.ReLU(True),
            nn.MaxPool2d(3,2)              # get [b,256,27,27]
        )
        # 定义第3个卷积层
        self.conv3 = nn.Sequential(
            nn.Conv2d(256, 384, 3, 1, 1),  # get [b,384,27,27]
            nn.ReLU(True),
            nn.MaxPool2d(3, 2)             # get [b,384,13,13]
        )
        # 定义第4个卷积层
        self.conv4 = nn.Sequential(
            nn.Conv2d(384, 384, 3, 1, 1),  # get [b,384,13,13]
            nn.ReLU(True),
        )
        # 定义第5个卷积层
        self.conv5 = nn.Sequential(
            nn.Conv2d(384, 256, 3, 1, 1),  # get [b,256,13,13]
            nn.ReLU(True),
            nn.MaxPool2d(3, 2)             # get [b,256,6,6]
        )
        # 定义6,7,8层全连接层
        self.dense = nn.Sequential(
            nn.Linear(256*6*6, 4096),      # get [b,4096]，需先reshape[b,9216]
            nn
            .ReLU(True),
            nn.Dropout(0.5),               # dropout减少正则化
            nn.Linear(4096, 4096),         # get [b,4096]
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, 1000)          # get [b,1000]
        )
    def forward(self, x):
        conv1_out = self.conv1(x)
        conv2_out = self.conv2(conv1_out)
        conv3_out = self.conv3(conv2_out)
        conv4_out = self.conv4(conv3_out)
        conv5_out = self.conv5(conv4_out)
        dense_input = conv5_out.view(conv5_out.size(0), -1)  # reshape 为一维
        out = self.dense(dense_input)
        return out

In [59]:
%%time
AlexNet_model = AlexNet()
# print(model)
data_random = torch.Tensor(range(3*227*227))
input = data_random.reshape([1,3,227,227])
print('1.随机输入的数据:', input.shape)
output = AlexNet_model(Variable(input))
print('2.模型输出:', output.shape)

1.随机输入的数据: torch.Size([1, 3, 227, 227])
2.模型输出: torch.Size([1, 1000])
Wall time: 563 ms


### 2.3 VGGNet
ImageNet 2014年亚军，使用小滤波器和更深层网络

VGG16网络表格参数设置(与函数对应)：
- 卷积 [in_channels,out_channels,kernel_size,stride,padding]

- 池化 [kernel_size,stride,padding]

|结构|参数设置|数据维度|
|-|-|-|
|input||[batch_size,3,224,224]|
|conv1|[3,64,3,1,padding=1]   |[batch_size,64,224,224]|
|conv2|[64,64,3,1,padding=1]  |[batch_size,64,224,224]|
|MaxPool|[2,2]                |[batch_size,64,112,112]|
|conv3|[64,128,3,1,padding=1] |[batch_size,128,112,112]|
|conv4|[128,128,3,1,padding=1]|[batch_size,128,112,112]|
|MaxPool|[2,2]                |[batch_size,128,56,56]|
|conv5|[128,256,3,1,padding=1]|[batch_size,256,56,56]|
|conv6|[256,256,3,1,padding=1]|[batch_size,256,56,56]|
|conv7|[256,256,3,1,padding=1]|[batch_size,256,56,56]|
|MaxPool|[2,2]                |[batch_size,256,28,28]|
|conv8|[256,512,3,1,padding=1]|[batch_size,512,28,28]|
|conv9|[512,512,3,1,padding=1]|[batch_size,512,28,28]|
|conv10|[512,512,3,1,padding=1]|[batch_size,512,28,28]|
|MaxPool|[2,2]                 |[batch_size,512,14,14]|
|conv11|[512,512,3,1,padding=1]|[batch_size,512,14,14]|
|conv12|[512,512,3,1,padding=1]|[batch_size,512,14,14]|
|conv13|[512,512,3,1,padding=1]|[batch_size,512,14,14]|
|MaxPool|[2,2]                |[batch_size,512,7,7]|
|dense1|[512\*7\*7,4096]      |[batch_size,4096]|
|dense2|[4096,4096]           |[batch_size,4096]|
|dense3|[4096,1000]           |[batch_size,1000]|

小结:
- VGGNet中使用$3\times3$的卷积核，stride=1，padding=1，所以卷积完后图像的宽和高不变，仅通道数改变
- VGGNet中使用$2\times2$的最大池化核,stride=2，所以池化后图像的宽和高会减为一半，但通道数不会改变

In [66]:
# 定义VGG-16的网络结构
# 方法一，笨办法，逐层定义
class VGG(nn.Module):
    def __init__(self):
        super().__init__()                 # 输入 [b,3,224,224]
        # conv参数 [in_channels,out_channels,kernel_size,stride,padding]
        # maxpool参数 [kernel_size, stride]
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),     # get [b,64,224,224]
            nn.ReLU(True),
            nn.Conv2d(64, 64, 3, 1, 1),    # get [b,64,224,224]
            nn.ReLU(True),
            
            nn.MaxPool2d(2, 2),            # get [b,64,112,112]
            
            nn.Conv2d(64, 128, 3, 1, 1),   # get [b,128,112,112]
            nn.ReLU(True),
            nn.Conv2d(128, 128, 3, 1, 1),  # get [b,128,112,112]
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),            # get [b,128,56,56]
            
            nn.Conv2d(128, 256, 3, 1, 1),  # get [b,256,56,56]
            nn.ReLU(True),
            nn.Conv2d(256, 256, 3, 1, 1),  # get [b,256,56,56]
            nn.ReLU(True),
            nn.Conv2d(256, 256, 3, 1, 1),  # get [b,256,56,56]
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),            # get [b,256,28,28]
            
            nn.Conv2d(256, 512, 3, 1, 1),  # get [b,512,28,28]
            nn.ReLU(True),
            nn.Conv2d(512, 512, 3, 1, 1),  # get [b,512,28,28]
            nn.ReLU(True),
            nn.Conv2d(512, 512, 3, 1, 1),  # get [b,512,28,28]
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),            # get [b,512,14,14]
            
            nn.Conv2d(512, 512, 3, 1, 1),  # get [b,512,14,14]
            nn.ReLU(True),
            nn.Conv2d(512, 512, 3, 1, 1),  # get [b,512,14,14]
            nn.ReLU(True),
            nn.Conv2d(512, 512, 3, 1, 1),  # get [b,512,14,14]
            nn.ReLU(True),
            nn.MaxPool2d(2, 2)            # get [b,512,7,7]    
        )
        # 将卷积层reshape为一维向量
        self.classifier = nn.Sequential(
            nn.Linear(512*7*7, 4096),     # get [b,4096]
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),         # get [b,4096]
            nn.ReLU(True),
            nn.Dropout(0.5),
            nn.Linear(4096, 1000)         # get [b,1000]
        )
    def forward(self, x):
        conv_features = self.features(x)   # 卷积层得到特征
        # reshape 为一维向量
        tense_input = conv_features.view(conv_features.size(0), -1)
        output = self.classifier(tense_input)
        return output

In [67]:
%%time
VGG16_model = VGG()
# print(model)
data_random = torch.Tensor(range(3*224*224))
input = data_random.reshape([1,3,224,224])
print('1.随机输入的数据:', input.shape)
output = VGG16_model(Variable(input))
print('2.模型输出:', output.shape)

1.随机输入的数据: torch.Size([1, 3, 224, 224])
2.模型输出: torch.Size([1, 1000])
Wall time: 2.16 s


In [30]:
# 方法二，完整的定义VGG网络
# 定义VGGNet的4种结构
cfg = {
    'VGG11':[64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13':[64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16':[64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512,\
             512, 512, 'M'],
    'VGG19':[64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, \
             'M', 512, 512, 512, 512, 'M']    
}
class VGG(nn.Module):
    def __init__(self, vgg_name, num_classes=10):
        super().__init__()    # 输入 [b,3,224,224]
        self.features = self._make_layers(cfg[vgg_name])    # 卷积层提取特征
        self.classifier = nn.Sequential(
            # fc6
            nn.Linear(512*7*7, 4096),        # get [b,4096]
            nn.ReLU(True),
            nn.Dropout(),
            # fc7
            nn.Linear(4096, 4096),           # get [b,4096]
            nn.ReLU(True),
            nn.Dropout(),
            # fc8
            nn.Linear(4096, num_classes)     # get [b,num_classes]          
        )
        self._initialize_weights()           # 初始化权重！！！
    def forward(self, x):
        conv_features = self.features(x)     # 卷积提取特征 
        tense_input = conv_features.view(conv_features.size(0), -1)   # reshape为一维向量
        output = self.classifier(tense_input)
        return output
    
    # 生成网络的层信息
    def _make_layers(self, net_cfg):         
        layers = []    # 将网络的结构写入到列表中
        in_channels = 3
        for x in net_cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]    # 将结构append到列表中
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, stride=1, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x                # 令下一层的输入为上层的输出
        # 有了均值池化层，可以减少全连接层的个数！！！！！！
#         layers += [nn.AvgPool2d(kernel_size=1, stride=1)] 
        return nn.Sequential(*layers)          # *代表可变参数，列表用*，字典用**
    
    # 初始化网络权重
    def _initialize_weights(self):
        for m in self.modules():               # 访问网络的各个模块
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0]*m.kernel_size[1]*m.out_channels
                m.weight.data.normal_(0, math.sqrt(2.0 /n ))    # 卷积权重初始化方法
                if m.bias is not None:
                    m.bias.data.zero_()                         # 卷积偏置初始为0
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)              # BN层权重初始为1
                m.bias.data.zero_()                 # BN层偏置初始为0
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)      # fc层权重初始为高斯分布
                m.bias.data.zero_()                 # fc层偏置初始为0                

In [36]:
%%time
vgg16_net = VGG('VGG16')
# print(vgg16_net)
input = torch.randn([1,3,224,224])
output = vgg16_net(Variable(input))
print('1.网络的输出:', output.shape)

1.网络的输出: torch.Size([1, 10])
Wall time: 2.83 s


### 2.4 GoogLeNet
又称InceptionNet，2014年ImageNet冠军，参数比AlenNet少12倍

In [1]:
###

In [ ]:
batch_size = 64
learning_rate = 1e-2
num_epochs = 20

In [ ]:
# 1.读取数据及预处理
# Compose将各种预处理操作组合，ToTensor将图片处理为tensor，
# Normalize(mean, variance)正则化
data_tf = transforms.Compose([transforms.ToTensor(), \
                              transforms.Normalize([0.5], [0.5])])
# 下载MNIST手写数字训练集
train_dataset = datasets.MNIST(root='./data/cifar10', train=True, transform=data_tf, download=True)
test_dataset = datasets.MNIST(root='./data/cifar10', train=False, transform=data_tf)
# 创建数据迭代器便于训练模型
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)